<img src='./logo.png'><br>
ref: https://www.kaggle.com/mlg-ulb/creditcardfraud

In [ ]:
from google.colab import drive
drive.mount("/content/gdrive/")

In [ ]:
# ! 쓰면 리눅스명령어(?) 쓸 수 있다. , list : ls, 현재위치: pwd
! pwd
# /content로 나온다.


In [ ]:
import pandas as pd
train = pd.read_csv("/content/gdrive/MyDrive/credit_k/dataset/creditcard.csv")
train.head()

In [ ]:
# pip install 사용해서 패키지 다운 가능 -> 버전 업그레이드나 다운그레이드 하면 코랩 기본 설정 건드린 거라 에러 날 수 있다.

* 주성분분석으로 나온 데이터이다.
* 고차원 데이터 -> 저차원 데이터로
* 주성분: 공분산 행렬의 고유벡터 ->직교함

In [ ]:
# $$$$
# 추가부분

In [ ]:
# 분석
import pandas as pd
import numpy as np

# 경고
import warnings
warnings.filterwarnings(action='ignore')

# 날짜 가공
import datetime as dt 
from dateutil.rrule import rrule, YEARLY, MONTHLY, WEEKLY

# 데이터 시각화
import matplotlib.pyplot as plt
plt.rc('font', family='Gothic') # For Windows. PLOT에서 한글 처리 -> 플롯 만드는 창에 넣어야 됨.

plt.rcParams['font.family']= 'Malgun Gothic'
plt.rcParams['axes.unicode_minus'] = False # - 기호 안 깨지게
plt.rcParams['font.size'] = 15
# plt.figure(figsize=(12,4)) # plot 12:4 비율

import seaborn as sns
sns.set() # seaborn에서 회색 grid 깔아줌
sns.set(rc={'figure.figsize':(18, 5)}) # 결과 plot 크기 키워줌

In [ ]:
# 사이킷런
from sklearn.model_selection import train_test_split

from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import LinearSVC

from sklearn.metrics import accuracy_score

In [ ]:
# 사이킷런 점수
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, precision_score, recall_score, f1_score
						 # ---> y_true, y_pred
from sklearn.metrics import precision_recall_curve # y_true, probas_pred
from sklearn.metrics import roc_auc_score, roc_curve # y_true, y_score

# 사이킷런 스케일러
from sklearn.preprocessing import StandardScaler, RobustScaler, MinMaxScaler

# 사이킷런 학습 데이터 증강
# StratifiedFold는 회귀에서는 X
from sklearn.model_selection import cross_val_score, KFold, StratifiedKFold, GridSearchCV

# Data load & pre-setting

## Data load

In [ ]:
# train=pd.read_csv('./dataset/creditcard.csv')

In [ ]:
train.head()

In [ ]:
train.shape

In [ ]:
train.info()

## Pre-Setting

### lower

In [ ]:
train.columns=[col.lower() for col in train.columns]
train.columns

# EDA

## target feature : imbalancing

In [ ]:
# 전체 비율
train['class'].value_counts()/(train.shape[0])*100

In [ ]:
# 시각화
print(train['class'].head())
train['class'].value_counts().sort_index().plot(kind='bar')

### 결론: imbalancing 하다

## unique 값 확인

In [ ]:
train.nunique()

### 결과
<pre>
<font size=3>
:: **target feature**: ""class""  
:: **class** :         2  
:: **amount** :        32767  
:: **time** :          124592  
</font>

## 상관계수 분석
:: null 없고 모두 float이라 바로 진행

### abs로 절댓값 큰 것 확인

In [ ]:
# sns.heatmap(crd.corr(), cmap='Blues' , annot=True, fmt=".2f")
sns.set(rc={'figure.figsize':(30, 20)})
sns.heatmap(abs(train.corr()), cmap='Blues' , annot=True, fmt=".2f")

#### 결론
<pre>
    절대값 기준
    amount와 v2 : 0.53
    amount와 v5 : 0.39
    amount와 v7 : 0.40
    amount와 v20 : 0.34
    time과 v3 : 0.42

### 양수 음수 관계 확인

In [ ]:
sns.set(rc={'figure.figsize':(30, 20)})
sns.heatmap(train.corr(), cmap='Blues' , annot=True, fmt=".2f")

### 상삼각행렬로 뽑아오기(코드 참고)
찾아보면 빨강 파랑으로도 할 수 있다.

In [ ]:
# $$$$
# np.triu 부분
# 탑 5, low 5 가져와도 가능

#### 결론
<pre>
    상관계수
    amount와 v2 : -0.53
    amount와 v5 : -0.39
    amount와 v7 : 0.40
    amount와 v20 : 0.34
    time과 v3 : -0.42

## 변수 distribution 분석

In [ ]:
for col in train.columns:
    sns.displot(train, x=col, kind="kde")
    sns.displot(train[train['class']==1], x=col, kind="kde")

In [ ]:
train[train['class']==1].head()

In [ ]:
# $$$$
train.describe()

# amount: 이상치 겁나 크다(왜도) -> 스케일링 해야 좋을듯??

In [ ]:
# $$$$ seaborn이나 matplot은 겁나 오래 걸리니까 df자체 기능 사용 추천
plt.figure(figsize=(16,10))
train.hist()
plt.show()

# time: 뭔가 쌍봉 느낌의 분포 -> 뭔가 결제 시간이 중앙에 몰린 느낌

# 1차 검정

## 함수

In [ ]:
# target이 multi일 경우
# Target is multiclass but average='binary'. Please choose another average setting, one of [None, 'micro', 'macro', 'weighted'].
# 너무 오래 걸려서 decision tree만 열어두겠다!!!

def my_score(y_true, y_pred):
    # report=classification_report(y_true, y_pred)
    # accuracy=accuracy_score(y_true, y_pred)
    
    cur_report=confusion_matrix(y_true,y_pred)
    print(cur_report)
    accuracy=accuracy_score(y_true, y_pred)
    
    precision=precision_score(y_true, y_pred,    average='binary') # 보통 macro로 써서 macro로 수정
    recall=recall_score(y_true, y_pred,          average='binary')
    f1=f1_score(y_true, y_pred,                  average='binary')
    # print('--'*30)
    # print("<REPORT>")
    # print(report, '\n')
    print('--'*30)
    print("<DETAIL>")
    print(f"\t accuracy:\t{accuracy:.4f} \n \t precision:\t{precision:.4f} \n \t recall:\t{recall:.4f} \n \t f1:\t\t{f1:.4f}\n")
    

        
def my_curv(y_test, proba_1):
    precision, recall, th = precision_recall_curve(y_test, proba_1) # proba 양일 때만 온다.
    plt.plot(th, precision[:len(th)], label="precision")
    plt.plot(th, recall[:len(th)], label="recall")
    plt.xlabel("th")
    plt.ylabel("score")
    plt.legend()
    plt.show()
    
    fpr, tpr, th=roc_curve(y_test, proba_1)
    plt.title(f"AUC Score : {roc_auc_score(y_test, proba_1):.4f}")
    plt.plot(fpr, tpr)
    plt.plot([0,1],[0,1],linestyle="--")
    plt.xlabel("FPR")
    plt.xlabel("TPR")
    plt.show()
    
    
# binary_mode: curve곡선은 binary_mode일 때만 고르겠다.
def my_fit_score(train, target="class", binary_mode=False):
    model_list = [
              # RandomForestClassifier(n_estimators=30, random_state=1234) , # n_estimators는 너무 느려서 일단 30으로 줄이겠다.
              # LogisticRegression(penalty="l2", random_state=1234),
              # KNeighborsClassifier(),
              DecisionTreeClassifier(), # tree 한 개
              #LinearSVC()ㄴ
             ]
    
    y = train[target]
    X = train.drop([target], axis=1)

    X_train,X_test, y_train, y_test = train_test_split(X, y, 
                                                   test_size=0.2, 
                                                   random_state=1234,
                                                   shuffle=True
                                                   )
   
    for model in model_list:
        model.fit(X_train, y_train)  #---학습해라
        pred = model.predict(X_test)
        # acc_score = accuracy_score(y_test, pred)
        print('=='*30)
        print(f"\t\t<<{model.__class__.__name__}>>")
        my_score(y_test, pred)
        
        #  print(model.feature_importances_) # 어레이로 출력됨. => 모델이 어떤 변수를 중요하게 봤나
        # -------------------------피쳐 중요도
        # plt.rc('font', family='Malgun Gothic')
        # plt.xticks(rotation=90) # 90도로 돌림
        # plt.bar(X.columns, model.feature_importances_)
        
        if binary_mode == True:
            proba=model.predict_proba(X_test)
            my_curv(y_test, proba[:, 1])
        
    print('=='*30)

#

## 함수 실행

In [ ]:
my_fit_score(train, target='class')

## 1차 점수

<pre>
============================================================
		<`RandomForestClassifier>
------------------------------------------------------------
<DETAIL>
	 accuracy:	0.9995 
 	 precision:	0.9467 
 	 recall:	0.7245 
 	 f1:		0.8208
        
============================================================
		<`LogisticRegression>
------------------------------------------------------------
<DETAIL>
	 accuracy:	0.9988 
 	 precision:	0.6495 
 	 recall:	0.6429 
 	 f1:		0.6462

============================================================
		<`KNeighborsClassifier>
------------------------------------------------------------
<DETAIL>
	 accuracy:	0.9984 
 	 precision:	1.0000 
 	 recall:	0.0510 
 	 f1:		0.0971

============================================================

### 결론  
<pre>
초기 params 설정:  
RandomForestClassifier(n_estimators=30, random_state=1234)  
-> (n_estimators는 너무 느려서 일단 30)  
LogisticRegression(penalty="l2", random_state=1234)  

acuuracy 점수 제외(imbalancing해서 나머지 의미 x)  
KNeighborsClassifier 모델 제외(튜닝할 거라 해도 점수 너무 낮음, 그리고 오래 걸림)  

# 변수 조정  
<pre>
 상관계수
    amount와 v2 : -0.53
    amount와 v5 : -0.39
    amount와 v7 : 0.40
    amount와 v20 : 0.34
    time과 v3 : -0.42
-> v2 삭제

In [ ]:
train2=train.drop(['v2'], axis=1)

In [ ]:
my_fit_score(train2, target='class')

## 2차 점수
<pre>
============================================================
		RandomForestClassifier
------------------------------------------------------------
<DETAIL>
	 accuracy:	0.9994 
 	 precision:	0.9103 
 	 recall:	0.7245 
 	 f1:		0.8068
============================================================
		KNeighborsClassifier
------------------------------------------------------------
<DETAIL>
	 accuracy:	0.9984 
 	 precision:	1.0000 
 	 recall:	0.0510 
 	 f1:		0.0971

============================================================

### 결론: drop 안 한다

# 튜닝

In [ ]:
y = train['class']
X = train.drop(['class'], axis=1)

In [ ]:
# y = train['class']
# X = train.drop(['class'], axis=1)
# X_train,X_test, y_train, y_test = train_test_split(X, y, 
#                                                    test_size=0.2,
#                                                    random_state=1234,
#                                                    shuffle=True
#                                                       )

# model = RandomForestClassifier(n_estimators=100, random_state=1234)
# #-- 튜닝 : 하이퍼파라미터 값을 임의로 변경해서 모델의 성능을 좋게 한다.
# myparam = {'n_estimators':[50, 100]
#         }

# skfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=1234) 
# gcv_model = GridSearchCV(model, param_grid=myparam, scoring="f1_macro", refit=True, cv=skfold)


# gcv_model.fit(X_train, y_train)
# print(gcv_model.best_score_)
# print(gcv_model.best_params_)
# print(gcv_model.cv_results_)



In [ ]:
# macro로 하니까 binary보다 점수 잘 나왔다. 왜일까?????
# pred = gcv_model.predict(X_test)
# f1 = f1_score(y_test, pred, average='macro')
# print(f"GCV f1 점수 {f1:.4f}") 

## ******** 확인 필요

In [ ]:
# y = train2['class']
# X = train2.drop(['class'], axis=1)
# X_train,X_test, y_train, y_test = train_test_split(X, y, 
#                                                    test_size=0.2,
#                                                    random_state=1234,
#                                                    shuffle=True
#                                                       )

# pred = gcv_model.predict(X_test)
# f1 = f1_score(y_test, pred, average='macro')
# print(f"GCV f1 점수 {f1:.4f}") 


# 시간 오래 걸릴 것 같아서 model 위에 거 가져오려고 따로 설정 안 했는데 아래와 같은 오류 메세지가 떴다.
# X has 29 features, but RandomForestClassifier is expecting 30 features as input.
# 그러면 feature 어떻게 반영할 건지에 대한 것도 들어가나???

In [ ]:
# 이렇게 해도 마찬가지네???
# y = train2['class']
# X = train2.drop(['class'], axis=1)
# X_train,X_test, y_train, y_test = train_test_split(X, y, 
#                                                    test_size=0.2,
#                                                    random_state=1234,
#                                                    shuffle=True
#                                                       )
# model = RandomForestClassifier(n_estimators=100, random_state=1234)
# pred = gcv_model.predict(X_test)
# f1 = f1_score(y_test, pred, average='macro')
# print(f"GCV f1 점수 {f1:.4f}") 

# X has 29 features, but RandomForestClassifier is expecting 30 features as input.

In [ ]:
# $$$$ 너무 오래 걸리면 StratifiedKFold에서 i==1이면 break하게 해서 데이터 1/10으로 줄이기

# imbalance 처리

In [ ]:
# ! pip install imblearn

In [ ]:
from imblearn.over_sampling import SMOTE     # 클래스
from imblearn.under_sampling import NearMiss # 클래스

In [ ]:
smote       =SMOTE(random_state=1234)
nearmiss    =NearMiss()

In [ ]:
over_X, over_y = smote.fit_resample(X,y)

## ROC, AUc 구하기

In [ ]:
y = train['class']
X = train.drop(['class'], axis=1)

In [ ]:
X_train,X_test, y_train, y_test = train_test_split(X, y, 
                                                   test_size=0.2, 
                                                   random_state=1234,
                                                   shuffle=True
                                                   )
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

### gcv_model로 ROC

In [ ]:
# gcv_model -> 이거 오버피팅 난 거 아님?
# pred = gcv_model.predict(X_test)
# proba = gcv_model.predict_proba(X_test)

In [ ]:
# pred = gcv_model.predict(X_test)
# proba = gcv_model.predict_proba(X_test)

# fpr, tpr, th = roc_curve(y_test, proba[:, 1])
# print(fpr[:5],'\n', tpr[:5], '\n',th[:5])
# print(len(fpr), len(tpr), len(th))

# sns.set(rc={'figure.figsize':(5, 3)})
# plt.figure()
# plt.xlabel("FPR - False Positive")
# plt.ylabel("TPR - True Positive")
# plt.plot(fpr, tpr,     "r",   label='rate')
# plt.plot([0,1], [0,1]  ,"b",  linestyle="--", label='random')
# plt.legend()
# plt.show()

In [ ]:
# proba = gcv_model.predict_proba(X_test[:10])
# print("model.predict_proba() \n", proba)
# print(  (proba[:,1]>0.5))
# print(  (proba[:,1]>0.5).astype(int))

# print(  (proba[:,1]>0.47).astype(int))

In [ ]:
# model
model = DecisionTreeClassifier()
# model = RandomForestClassifier(n_estimators=100, random_state=1234)
model.fit(X_train, y_train)
pred = model.predict(X_test)
proba = model.predict_proba(X_test)

In [ ]:
fpr, tpr, th = roc_curve(y_test, proba[:, 1])
print(fpr[:5],'\n', tpr[:5], '\n',th[:5])
print(len(fpr), len(tpr), len(th))

sns.set(rc={'figure.figsize':(5, 3)})
plt.figure()
plt.xlabel("FPR - False Positive")
plt.ylabel("TPR - True Positive")
plt.plot(fpr, tpr,     "r",   label='rate')
plt.plot([0,1], [0,1]  ,"b",  linestyle="--", label='random')
plt.legend()
plt.show()

In [ ]:
# proba = gcv_model.predict_proba(X_test[:10])
# print("model.predict_proba() \n", proba)
# print(  (proba[:,1]>0.5))
# print(  (proba[:,1]>0.5).astype(int))

# print(  (proba[:,1]>0.47).astype(int))

In [ ]:
my_fit_score(train, 'class', binary_mode=True)

# 전처리, 가공

## Outlier(이상치) 제거
* Target feature인 class에 영향 많이 미치는 변수 중에 아웃라이어 있는 애들 제거해준다.
* Target Class (+) : v11, v4, v2, v21
* Target Class (-) : v17 v14 v10 v16

In [ ]:
Q1 = train['v11'].quantile(q=0.25)
Q3 = train['v11'].quantile(q=0.75)
print(Q1, Q3)
IQR = Q3-Q1
print(Q1, Q3, IQR)

lower_lm=Q1-1.5*IQR
upper_lm=Q3+1.5*IQR
print(lower_lm, upper_lm)

In [ ]:
# 지우기 연습!
train['v11'][(train['v11']>upper_lm) | (train['v11']<lower_lm)]
drop_idx = train['v11'][(train['v11']>upper_lm) | (train['v11']<lower_lm)].index
train.drop(drop_idx, axis=0) # 실제 반영은 안 할 거라 변수에 적용 안 했음

## 눈으로 확인해서 지우기

### 박스플롯 그리기

In [ ]:
sns.set_theme(style="whitegrid") # 예쁘게 하는 용이라 필요 없다.

ax = sns.boxplot(x=train["v11"])

In [ ]:
# 지우기 연습
train['v11'][(train['v11']>7.5) | (train['v11']<-4)] # 80개
drop_idx = train['v11'][(train['v11']>7.5) | (train['v11']<-4)].index # tolist()해도됨
drop_idx
# train.drop(drop_idx, axis=0) # 실제 반영은 안 할 거라 변수에 적용 안 했음

# 이상치 제거 후 점수 봐보자

In [ ]:
train['v11'][(train['v11']>upper_lm) | (train['v11']<lower_lm)]
drop_idx = train['v11'][(train['v11']>upper_lm) | (train['v11']<lower_lm)].index
train2=train.drop(drop_idx, axis=0)

In [ ]:
my_fit_score(train2, 'class')

In [ ]:
# 지우기 연습
train['v11'][(train['v11']>7.5) | (train['v11']<-4)] # 80개
drop_idx = train['v11'][(train['v11']>7.5) | (train['v11']<-4)].index # tolist()해도됨
train3=train.drop(drop_idx, axis=0)

In [ ]:
my_fit_score(train3, 'class')